In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import keras
from sklearn.preprocessing import MinMaxScaler


In [23]:
file_path = 'https://docs.google.com/spreadsheets/d/1V6cBNRohsvWOEcPUMWyZ6tMAvc25Ji2U/export?format=csv'
data = pd.read_csv(file_path)



In [24]:

# Ensure all values are numeric (convert non-numeric values to NaN)
data = data.apply(pd.to_numeric, errors='coerce').fillna(0)
# Remove rows where all values are zero
data = data.loc[~(data.eq(0).all(axis=1))]

data.tail(30)



,time,max_seen,min_seen,trend_type,price,leg_3_to_1,leg_2_to_1,leg_2_correction,resistance_filter,rsi_filter,resi_value,max_move
19602,0.0,1.08191,1.08088,1,1.08173,1.37,0.33,71,0,0,74.60,18
19603,0.0,1.08191,1.08108,-1,1.08111,0.60,0.63,38,1,1,32.46,3
19604,0.0,1.08145,1.08109,-1,1.08111,0.46,0.44,87,1,0,27.67,2
19605,0.0,1.08204,1.08056,-1,1.08118,0.85,0.58,86,1,0,26.32,62
19606,0.0,1.08263,1.08052,1,1.08167,4.13,0.96,63,1,1,66.87,96
19607,0.0,1.08263,1.07688,-1,1.07975,0.80,0.18,24,0,1,38.69,287
19608,0.0,1.08590,1.08099,1,1.08175,0.40,0.06,36,0,1,68.59,415
19609,0.0,1.08590,1.08114,1,1.08163,1.25,0.47,41,0,1,62.32,427
19610,0.0,1.08263,1.08130,-1,1.08181,1.64,0.43,96,0,1,35.31,51
19611,0.0,1.08259,1.08130,-1,1.08172,0.94,0.22,23,0,1,34.15,42


In [25]:
# # replace all negative values in max_price_move with 0 because out stoploss has been hitted anyway
# data["max_price_move"] = data["max_price_move"].apply(lambda x: max(0, x))
# data

In [26]:
reserve = data
data = data.drop(['time','max_seen', 'min_seen'], axis=1)


In [17]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19632 entries, 0 to 19631
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   time               19632 non-null  float64
 1   max_seen           19632 non-null  float64
 2   min_seen           19632 non-null  float64
 3   trend_type         19632 non-null  int64  
 4   price              19632 non-null  float64
 5   leg_3_to_1         19632 non-null  float64
 6   leg_2_to_1         19632 non-null  float64
 7   leg_2_correction   19632 non-null  int64  
 8   resistance_filter  19632 non-null  int64  
 9   rsi_filter         19632 non-null  int64  
 10  resi_value         19632 non-null  float64
 11  max_move           19632 non-null  int64  
dtypes: float64(7), int64(5)
memory usage: 1.9 MB


In [ ]:
data.describe()

In [ ]:
data.shape

In [ ]:
data.isnull().sum()

In [ ]:
# normalized_df = keras.utils.normalize(data)
# type(normalized_df)
# data = pd.DataFrame(normalized_df, columns=["leg_3_to_leg_1_time","leg_2_to_leg_1_time","leg_2_correction_percent","resistance","rsi_filter_passed","normalized_rsi","max_price_move"])
# data

In [ ]:
#normalize data
columns_to_normalize = data.columns.difference(['max_price_move'])
scaler = MinMaxScaler()
data[columns_to_normalize] = scaler.fit_transform(data[columns_to_normalize])
print(data)



In [ ]:

# Separate features and target variable
X = data.drop('max_move', axis=1)
y = data['max_move']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize a StandardScaler for scaling the features
scaler = StandardScaler()

# Fit the scaler on the training data and transform both training and testing sets
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Define the model
model = Sequential([
    Dense(64, activation='sigmoid', input_shape=(X_train_scaled.shape[1],)),
    Dense(32, activation='sigmoid'),
    Dense(16, activation='sigmoid'),
    Dense(1)  # Output layer for regression
])

# Compile the model with mean squared error loss for regression
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Train the model
history = model.fit(X_train_scaled, y_train, epochs=50, batch_size=32, validation_split=0.2, verbose=1)

# Evaluate the model on the test set
test_loss, test_mae = model.evaluate(X_test_scaled, y_test, verbose=1)
print(f"Test MAE: {test_mae}")

# Predict on new data (example)
predictions = model.predict(X_test_scaled)


In [ ]:
import matplotlib.pyplot as plt

# Plot training & validation loss values
plt.figure(figsize=(12, 5))

# Loss
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss (MSE)')
plt.legend()

# Mean Absolute Error
plt.subplot(1, 2, 2)
plt.plot(history.history['mae'], label='Training MAE')
plt.plot(history.history['val_mae'], label='Validation MAE')
plt.title('Model Mean Absolute Error')
plt.xlabel('Epoch')
plt.ylabel('MAE')
plt.legend()

plt.show()


In [ ]:
test_loss, test_mae = model.evaluate(X_test_scaled, y_test, verbose=1)
print(f"Test Loss (MSE): {test_loss}")
print(f"Test Mean Absolute Error (MAE): {test_mae}")


In [ ]:
import numpy as np

# Make predictions
predictions = model.predict(X_test_scaled).flatten()

# Compare predictions with actual values
comparison_df = pd.DataFrame({'Actual': y_test, 'Predicted': predictions})
print(comparison_df.head(10))  # Display first 10 rows for a quick check
